In [22]:
from bs4 import BeautifulSoup
import cloudscraper
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import pandas as pd

In [17]:
scraper = cloudscraper.create_scraper()  # returns a CloudScraper instance
html = scraper.get("https://www.hemnet.se/salda/bostader?item_types%5B%5D=villa&page=1&sold_age=12m").content
soup = BeautifulSoup(html, 'html.parser')

In [11]:
def get_house_links(soup):
    """Takes the soup of a page and returns a list of links to each house"""
    a_tags = soup.find_all('a',class_='sold-property-link js-sold-property-card-link')
    return [tag.get('href') for tag in a_tags]

def get_houses_info(link:list[str]):
    """Takes the link to a house and returns a dictionary with the house properties"""
    
    


In [ ]:
scraper_houses = cloudscraper.create_scraper()  # returns a CloudScraper instance
links = ['hemnet.se/salda/villa-5rum-solbacken-malmo-kommun-vilebovagen-22a-207996096575631945']
html = scraper.get(links).content
soup = BeautifulSoup(html, 'html.parser')




In [21]:


def get_coordinates(addresses:list[str]):
    geolocator = Nominatim(user_agent="House_price_prediction")
    geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)
    
    locations = []
    
    for address in addresses:
        location = geolocator.geocode(address)
        locations.append((location.latitude, location.longitude))



(57.7079695, 12.8036417)
